# dynoNet: Exercise
## Application to Ground Vibration Test of an F-16 aircraft. Description of the benchmark: https://www.nonlinearbenchmark.org/benchmarks/f-16-gvt

### Course on Deep Learning for System Identification
### Authors: Dario Piga, Marco Forgione
### Milano, July 3rd, 2035

Repeat the same exercise on the identification of the F16 aircraft.  **Use a dynoNet** instead of Feedforward and RNN networks.

Hyper-parameter to select:
* structure of the network 
* input and output lags of the LTI blocks
* structure of the Static Nonlinearities
* sub-sequence length for training and batch size
* number of initial samples skipped in the creation of the loss 
* optimizer's hyper-parameters (learning rate, maximum number of epochs)

In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import dynonet
from dynonet.lti import MimoLinearDynamicalOperator
from dynonet.static import MimoStaticNonLinearity
from sklearn.preprocessing import StandardScaler
import pandas as pd
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim
from sklearn.metrics import r2_score
from torch.utils.data import Dataset, DataLoader

In [2]:
# %matplotlib widget 

Since dynoNet receives as an input the input sequence of the system, the same *DataSet* class *F16DS_seq* used for training RNN can be also used in this exercise. Each element of the *F16DS_seq* Dataset is a pair with:
- input sub-sequences of a given length (since we are considering two inputs, we have 2 input sub-sequences)
- corresponding output sequences (if you consider all the three outputs, you will have 3 outputs)

For your convenience, you can run the cell below to upload the dataset and instantiate the *F16DS_seq* class.

In [3]:
folder = os.path.join("..", "..", "data", "F16")
filename_train = os.path.join(folder, "F16Data_SineSw_Level3.csv")
filename_test = os.path.join(folder, "F16Data_SineSw_Level4_Validation.csv")

In [4]:
df_train = pd.read_csv(filename_train)
df_test = pd.read_csv(filename_test)

In [5]:
# data normalization
ds_mean = df_train.mean()
ds_std = df_train.std()
df_train = df_train - ds_mean / ds_std
df_test = df_test - ds_mean / ds_std

In [6]:
output_vars = ["Acceleration1"]
input_vars = ["Force", "Voltage"]
seq_len = 10_000

In [7]:
from torch.utils.data import Dataset

class SequenceDataset(Dataset):
    r"""A dataset returning sub-sequences extracted from longer sequences.
        For simplicity, this version does not support overlapping subsequences.
    Args:
        *tensors (Tensor): tensors that have the same size on the first dimension.
    Examples:
        >>> u = torch.randn(1000, 2) # 2 inputs
        >>> y = torch.randn(1000, 3) # 3 outputs
        >>> train_dataset = SequenceDataset(u, y, seq_len=100)
    """

    def __init__(self, *tensors, seq_len):
        self.tensors = tensors

        self.seq_len = seq_len
        assert all(tensor.shape[0] == self.tensors[0].shape[0] for tensor in self.tensors), "All tensors must have the same length"
        self.total_len = self.tensors[0].shape[0]

    def __len__(self):
        return int(self.total_len // self.seq_len)

    def __getitem__(self, idx):
        start = idx * self.seq_len
        stop = start + self.seq_len
        return [tensor[start:stop] for tensor in self.tensors]

In [8]:
# Datasets
train_ds = SequenceDataset(
    torch.Tensor(df_train[input_vars].values).type(torch.Tensor),
    torch.Tensor(df_train[output_vars].values).type(torch.Tensor),
    seq_len=seq_len
)

... # continue with the test dataset

Ellipsis

In [9]:
len(train_ds)

10